In [2]:
import os
import glob
import subprocess
import pandas as pd

# Filter sequences

In [3]:
input = "tlr4-anta1"
best_binders = pd.read_csv(f"output/{input}/af2_best.csv")
all_binders = pd.read_csv(f"output/{input}/af2_results.csv")
best_binders

,mpnn,plddt,i_ptm,i_pae,rmsd,seq,model_path,binder-rmsd,input_pdb
0,1.240117,0.767540,0.607834,13.748378,2.626005,RLHKLTLRNNFDSLNVMKTCIQGLAGLEVHRLVLGEFRNEGNLEKF...,output/tlr4-anta1/mpnn_af2/tlr4-anta1_3H_3_1/a...,2.002460,output/tlr4-anta1/rf_dock/tlr4-anta1_3H_3_1/th...
1,1.183955,0.746155,0.572466,14.940110,2.991684,RLHKLTLRNNFDSLNVMKTCIQGLAGLEVHRLVLGEFRNEGNLEKF...,output/tlr4-anta1/mpnn_af2/tlr4-anta1_3H_3_1/a...,2.651465,output/tlr4-anta1/rf_dock/tlr4-anta1_3H_3_1/th...
2,1.178961,0.845178,0.749347,9.913550,1.380248,RLHKLTLRNNFDSLNVMKTCIQGLAGLEVHRLVLGEFRNEGNLEKF...,output/tlr4-anta1/mpnn_af2/tlr4-anta1_3H_3_1/a...,2.332067,output/tlr4-anta1/rf_dock/tlr4-anta1_3H_3_1/th...
3,1.132551,0.679869,0.505421,17.004433,1.531126,RLHKLTLRNNFDSLNVMKTCIQGLAGLEVHRLVLGEFRNEGNLEKF...,output/tlr4-anta1/mpnn_af2/tlr4-anta1_3H_3_1/a...,2.736440,output/tlr4-anta1/rf_dock/tlr4-anta1_3H_3_1/th...
4,1.082235,0.833184,0.723206,10.593599,1.657610,RLHKLTLRNNFDSLNVMKTCIQGLAGLEVHRLVLGEFRNEGNLEKF...,output/tlr4-anta1/mpnn_af2/tlr4-anta1_3H_3_1/a...,2.287386,output/tlr4-anta1/rf_dock/tlr4-anta1_3H_3_1/th...
5,1.197225,0.804455,0.730521,10.738312,2.560902,RLHKLTLRNNFDSLNVMKTCIQGLAGLEVHRLVLGEFRNEGNLEKF...,output/tlr4-anta1/mpnn_af2/tlr4-anta1_3H_12_1/...,2.168069,output/tlr4-anta1/rf_dock/tlr4-anta1_3H_12_1/t...
6,1.172246,0.873073,0.799475,8.458550,1.404431,RLHKLTLRNNFDSLNVMKTCIQGLAGLEVHRLVLGEFRNEGNLEKF...,output/tlr4-anta1/mpnn_af2/tlr4-anta1_3H_12_1/...,2.237683,output/tlr4-anta1/rf_dock/tlr4-anta1_3H_12_1/t...
7,1.249368,0.780831,0.621569,13.631664,1.280700,RLHKLTLRNNFDSLNVMKTCIQGLAGLEVHRLVLGEFRNEGNLEKF...,output/tlr4-anta1/mpnn_af2/tlr4-anta1_3H_12_1/...,2.193000,output/tlr4-anta1/rf_dock/tlr4-anta1_3H_12_1/t...
8,1.225072,0.801374,0.757825,10.227311,1.678934,RLHKLTLRNNFDSLNVMKTCIQGLAGLEVHRLVLGEFRNEGNLEKF...,output/tlr4-anta1/mpnn_af2/tlr4-anta1_3H_12_1/...,2.176869,output/tlr4-anta1/rf_dock/tlr4-anta1_3H_12_1/t...
9,1.222576,0.891346,0.868585,7.095359,2.319043,RLHKLTLRNNFDSLNVMKTCIQGLAGLEVHRLVLGEFRNEGNLEKF...,output/tlr4-anta1/mpnn_af2/tlr4-anta1_3H_12_1/...,2.026261,output/tlr4-anta1/rf_dock/tlr4-anta1_3H_12_1/t...


In [7]:
filtered = best_binders[
    (best_binders["rmsd"] < 3)
    & (best_binders["binder-rmsd"] < 4)
    & (best_binders["plddt"] > 0.84)
]
# [(best_binders["plddt"]>0.7)&(best_binders["i_pae"]<8)&(best_binders["rmsd"]<3)]

filtered = filtered.sort_values(by="plddt", ascending=False)
filtered

,mpnn,plddt,i_ptm,i_pae,rmsd,seq,model_path,binder-rmsd,input_pdb
9,1.222576,0.891346,0.868585,7.095359,2.319043,RLHKLTLRNNFDSLNVMKTCIQGLAGLEVHRLVLGEFRNEGNLEKF...,output/tlr4-anta1/mpnn_af2/tlr4-anta1_3H_12_1/...,2.026261,output/tlr4-anta1/rf_dock/tlr4-anta1_3H_12_1/t...
6,1.172246,0.873073,0.799475,8.458550,1.404431,RLHKLTLRNNFDSLNVMKTCIQGLAGLEVHRLVLGEFRNEGNLEKF...,output/tlr4-anta1/mpnn_af2/tlr4-anta1_3H_12_1/...,2.237683,output/tlr4-anta1/rf_dock/tlr4-anta1_3H_12_1/t...
11,1.110579,0.865011,0.843117,8.040300,2.075729,RLHKLTLRNNFDSLNVMKTCIQGLAGLEVHRLVLGEFRNEGNLEKF...,output/tlr4-anta1/mpnn_af2/tlr4-anta1_3H_12_1/...,1.974150,output/tlr4-anta1/rf_dock/tlr4-anta1_3H_12_1/t...
2,1.178961,0.845178,0.749347,9.913550,1.380248,RLHKLTLRNNFDSLNVMKTCIQGLAGLEVHRLVLGEFRNEGNLEKF...,output/tlr4-anta1/mpnn_af2/tlr4-anta1_3H_3_1/a...,2.332067,output/tlr4-anta1/rf_dock/tlr4-anta1_3H_3_1/th...
10,1.217498,0.844223,0.698743,11.086200,2.005128,RLHKLTLRNNFDSLNVMKTCIQGLAGLEVHRLVLGEFRNEGNLEKF...,output/tlr4-anta1/mpnn_af2/tlr4-anta1_3H_12_1/...,2.226235,output/tlr4-anta1/rf_dock/tlr4-anta1_3H_12_1/t...


# Prepare folder of best binders

In [16]:
# Create output dir
binder_folder = "input"
output_dir = f"output/{input}/opt_binders"
binders_input = f"{output_dir}/{binder_folder}"
os.makedirs(f"{binders_input}", exist_ok=True)

# Copy binder inputs to new dir
pdbs = filtered["model_path"].to_list()
for pdb in pdbs:
    !cp $pdb $binders_input

# Count .pdb files in the folder
pdb_files = glob.glob(os.path.join(binders_input, "*.pdb"))
pdb_count = len(pdb_files)
print(f"Number of binders to optimize: {pdb_count}. Will run {pdb_count} arrays")

Number of binders to optimize: 4. Will run 4 arrays


# Run optimization for each binder in parallel

In [17]:
# Binder optimization parameters
iterations = 30
starting_mpnns = 200

designs_per_iteration = 10
mpnn_per_design = 5
sort = "i_pae"

af2_recycles = 12
mpnn_sampling_temp = 0.15

array_number = pdb_count
array_limit = 10

# Script arguments
bash_arguments = f"--output=/dev/null --array=0-{array_number-1}%{array_limit}"
script_arguments = f"{binders_input} {output_dir} {iterations} {starting_mpnns} {designs_per_iteration} {mpnn_per_design} {sort} {af2_recycles} {mpnn_sampling_temp}"

command = f"sbatch {bash_arguments} helper_scripts/binder_opt.sh {script_arguments}"

print(f"command will run {array_number} binder optimizations, each doing")
print(
    f"{iterations} iterations with {designs_per_iteration*mpnn_per_design} sampled designs."
)
print(
    f"Each script will return {iterations*designs_per_iteration*mpnn_per_design} designs ({iterations*designs_per_iteration*mpnn_per_design*array_number} in total)"
)
print(command)

command will run 4 binder optimizations, each doing
30 iterations with 50 sampled designs.
Each script will return 1500 designs (6000 in total)
sbatch --array=0-3%10 helper_scripts/binder_opt.sh output/tlr4-anta1/opt_binders/input output/tlr4-anta1/opt_binders 30 200 10 5 i_pae 12 0.15


In [22]:
# Run the array bash script
subprocess.run(command, shell=True)

Submitted batch job 145261


CompletedProcess(args='sbatch --array=0-3%10 helper_scripts/binder_opt.sh output/tlr4-anta1/opt_binders/input output/tlr4-anta1/opt_binders 30 200 10 5 i_pae 12 0.15', returncode=0)

In [24]:
!squeue --me

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
          145086_0       gpu binder_o  tsatler  R 2-03:42:44      1 compute-6-2
          145086_1       gpu binder_o  tsatler  R 2-03:42:44      1 compute-6-2
          145086_2       gpu binder_o  tsatler  R 2-03:42:44      1 compute-6-2
          145086_3       gpu binder_o  tsatler  R 2-03:42:44      1 compute-6-2
          145086_4       gpu binder_o  tsatler  R 2-03:42:44      1 compute-6-3
          145086_5       gpu binder_o  tsatler  R 2-03:42:44      1 compute-6-3
          145261_0       gpu binder_o  tsatler  R       1:52      1 compute-0-12
          145261_1       gpu binder_o  tsatler  R       1:52      1 compute-0-12
          145261_2       gpu binder_o  tsatler  R       1:52      1 compute-0-12
          145261_3       gpu binder_o  tsatler  R       1:52      1 compute-0-12
